## 5. Convert CSV to PARQUET File

Code learned and followed from AAI 540 Lab 2

In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
from pyathena import connect

In [4]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/Smart_Grid/data/parquet".format(bucket)

# Set Athena parameters
database_name = "dsoawsfp"
table_name_csv = "meterdata_csv"
table_name_parquet = "meterdata_parquet"

In [5]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [6]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [7]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (
    format = 'PARQUET',
    external_location = '{}',
    partitioned_by = ARRAY['year', 'month']
) AS
SELECT   timestamp,
         kWh,
         weekday,
         hour,
         temperature,
         Holiday,
         year,
         month
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_csv
)

print(statement)

CREATE TABLE IF NOT EXISTS dsoawsfp.meterdata_parquet
WITH (
    format = 'PARQUET',
    external_location = 's3://sagemaker-us-east-1-911199926915/Smart_Grid/data/parquet',
    partitioned_by = ARRAY['year', 'month']
) AS
SELECT   timestamp,
         kWh,
         weekday,
         hour,
         temperature,
         Holiday,
         year,
         month
FROM dsoawsfp.meterdata_csv


In [8]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_244/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


Empty DataFrame
Columns: [rows]
Index: []

In [9]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoawsfp.meterdata_parquet


In [10]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_244/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


Empty DataFrame
Columns: []
Index: []

In [11]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoawsfp.meterdata_parquet


In [12]:
df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

/tmp/ipykernel_244/2831825248.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_partitions = pd.read_sql(statement, conn)


partition
0   year=2012/month=7
1  year=2011/month=11
2   year=2012/month=5
3   year=2012/month=1
4  year=2012/month=12

In [21]:
year = int(2012)
month = 5

statement = """SELECT * FROM {}.{}
    WHERE year = 2014 LIMIT 100""".format(
    database_name, table_name_parquet
)

print(statement)

SELECT * FROM dsoawsfp.meterdata_parquet
    WHERE year = 2014 LIMIT 100


In [22]:
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_244/8222894.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


timestamp       kwh  weekday  hour  temperature  holiday  year  \
0  2014-01-01 00:00:00  0.259011        2   0.0         7.76      5.0  2014   
1  2014-01-01 03:00:00  0.158080        2   3.0         6.76      5.0  2014   
2  2014-01-01 04:00:00  0.140162        2   4.0         6.74      5.0  2014   
3  2014-01-01 05:00:00  0.131367        2   5.0         7.24      5.0  2014   
4  2014-01-01 06:00:00  0.134517        2   6.0         8.02      5.0  2014   
..                 ...       ...      ...   ...          ...      ...   ...   
95 2014-01-03 10:00:00  0.228568        4  10.0         8.36     -1.0  2014   
96 2014-01-03 11:00:00  0.232194        4  11.0         9.88     -1.0  2014   
97 2014-01-03 12:00:00  0.239475        4  12.0        10.14     -1.0  2014   
98 2014-01-03 13:00:00  0.262311        4  13.0         8.96     -1.0  2014   
99 2014-01-03 14:00:00  0.257353        4  14.0         7.93     -1.0  2014   

    month  
0       1  
1       1  
2       1  
3       1  
4       1  
..    ...  
95      1  
96      1  
97      1  
98      1  
99      1  

[100 rows x 8 columns]

In [23]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>